In [1]:
import pandas as pd
import json
import zipfile
from tqdm import tqdm
from textblob import TextBlob
import os
import glob

In [2]:
# Correct relative path from your notebook
zip_path = 'data/raw/Full_HotelRec.zip'

# List files inside the zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print(zip_ref.namelist())

['HotelRec.txt']


In [3]:
# --- Settings ---
txt_inside_zip = 'HotelRec.txt'                     # file inside ZIP
output_folder = 'data/raw2/'                        # where to save
partial_folder = 'data/raw2/partial/'               # For the parts
chunk_size = 1_000_000                              # how many lines per part
final_csv_path = 'data/raw2/hotelrec_full_combined.csv'  # <-- new final file path

In [4]:
# --- Prepare output folder ---
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(partial_folder):
    os.makedirs(partial_folder)

In [ ]:
# --- Count total lines ---
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(txt_inside_zip) as file:
        total_lines = sum(1 for _ in file)

print(f"Total lines in dataset: {total_lines}") #3m 48.4s

Total lines in dataset: 50264531


In [6]:
def parse_hotel_url(url):
    try:
        parts = url.split('-')
        hotel_id = parts[2]  # dXXXXX
        if hotel_id.startswith('d'):
            hotel_id = hotel_id[1:]  # remove leading 'd'
        
        hotel_name = parts[4].replace('_', ' ') if len(parts) > 4 else None
        hotel_location = parts[5].replace('_', ' ') if len(parts) > 5 else None
        
        # Remove '.html' if it's stuck at the end
        if hotel_location and hotel_location.endswith('.html'):
            hotel_location = hotel_location.replace('.html', '')
        
        return hotel_id, hotel_name, hotel_location
    except:
        return None, None, None

In [7]:
# --- Initialize ---
batch = []
file_counter = 1

In [8]:
starting_row = 0  # Set this to the row number you want to start from

In [ ]:
# --- Start processing ---
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(txt_inside_zip) as file:
        for i, line in enumerate(tqdm(file, total=total_lines, desc="Processing chunks", mininterval=1)):

            if i < starting_row:
                continue  # Skip lines until starting_row
            
            record = json.loads(line.decode('utf-8'))

            # Combine title and text
            title = record.get('title', '')
            text = record.get('text', '')
            combined_text = f"{title}. {text}"

            sentiment = TextBlob(combined_text).sentiment.polarity

            # Format date (keep only YYYY-MM)
            raw_date = record.get('date', None)
            if raw_date:
                formatted_date = raw_date.split('T')[0][:7]  # Take only 'YYYY-MM'
            else:
                formatted_date = None

            # Parse hotel_url
            hotel_id, hotel_name, hotel_location = parse_hotel_url(record.get('hotel_url', ''))

            # Create the record
            new_record = {
                'hotel_id': hotel_id,
                'hotel_name': hotel_name,
                'hotel_location': hotel_location,
                'date': formatted_date,
                'rating': record.get('rating'),
                'sentiment_score': sentiment,
            }

            # Flatten property_dict
            prop = record.get('property_dict', {})
            if prop:
                for key in ['sleep quality', 'value', 'rooms', 'service', 'cleanliness', 'location']:
                    new_record[key] = prop.get(key)

            batch.append(new_record)

            # Save every chunk_size lines
            if (i + 1) % chunk_size == 0:
                temp_df = pd.DataFrame(batch)
                part_filename = os.path.join(partial_folder, f'hotelrec_part{file_counter}.csv')
                temp_df.to_csv(part_filename, index=False)
                print(f"Saved part: {part_filename}")
                file_counter += 1
                batch = []

        # Save remaining at the end
        if batch:
            temp_df = pd.DataFrame(batch)
            part_filename = os.path.join(partial_folder, f'hotelrec_part{file_counter}.csv')
            temp_df.to_csv(part_filename, index=False)
            print(f"Saved final part: {part_filename}")

print("All parts saved!") #256m 39.1s

Processing chunks:   2%|████▌                                                                                                                                                                                                                                | 1000070/50264531 [05:08<8:39:19, 1581.05it/s]

Saved part: data/raw2/partial/hotelrec_part1.csv


Processing chunks:   4%|█████████                                                                                                                                                                                                                            | 1999377/50264531 [10:21<4:25:43, 3027.22it/s]

Saved part: data/raw2/partial/hotelrec_part2.csv


Processing chunks:   6%|█████████████▋                                                                                                                                                                                                                       | 2997519/50264531 [15:16<3:48:08, 3453.08it/s]

...

Saved part: data/raw2/partial/hotelrec_part49.csv


Processing chunks:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 50000748/50264531 [4:15:18<02:52, 1529.13it/s]

Saved part: data/raw2/partial/hotelrec_part50.csv


Processing chunks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50264531/50264531 [4:16:37<00:00, 3264.56it/s]


Saved final part: data/raw2/partial/hotelrec_part51.csv
All parts saved!


In [10]:
# Step 1: Find all parts
all_parts = sorted(glob.glob(os.path.join(partial_folder, 'hotelrec_part*.csv')))
print(f"Found {len(all_parts)} parts to combine.")

# Step 2: Read and combine
full_df = pd.concat((pd.read_csv(part) for part in all_parts), ignore_index=True)
full_df.to_csv(final_csv_path, index=False)

print(f"Successfully combined into: {final_csv_path}")
print(f"Final combined dataset shape: {full_df.shape}") # 2m 43.3s

Found 51 parts to combine.
Successfully combined into: data/raw2/hotelrec_full_combined.csv
Final combined dataset shape: (50264531, 12)


In [ ]:

full_df.to_csv(final_csv_path.replace('.csv', '.csv.gz'), index=False, compression='gzip') #5m 55.1s